# Excel Data Appender

This Notebook is for an interactive application to collate data from a source Excel file to a target Excel file. You can map columns between the two files to ensure data aligns correctly.

## How to Use
1. **Select Files**: Use the file choosers to pick your source and target Excel files.
2. **Map Columns**: Click 'Map Columns' to review and adjust how columns align.
3. **Append Data**: Confirm mappings and append the data, with results displayed below.

## Step 1: Import Necessary Libraries

Here, we bring in the tools we need:
- **`Ipython.display`**: Used for displying content in Jupyter notebooks
- **`ipywidgets`**: Creates interactive controls like buttons and dropdowns.
- **`ipyfilechooser`**: Provides a file picker for selecting Excel files.
- **`pandas`**: Handles reading and writing Excel data.
- **`fuzzywuzzy`**: Suggests column matches when names aren’t exact.

**Note**: You’ll need these libraries installed. Run these commands in your terminal if you haven’t:
```bash
pip install ipywidgets ipyfilechooser pandas fuzzywuzzy
```
For faster fuzzy matching, optionally install:
```bash
pip install python-levenshtein
```

In [ ]:
from IPython.display import display, HTML
import ipywidgets as widgets
from ipyfilechooser import FileChooser
import pandas as pd
from fuzzywuzzy import process

## Step 2: Define the Append Data Function

This function reads the Excel files, applies your column mappings, appends/collates the data, and saves the result.

### What It Does
- **Reads Files**: Loads data from both Excel files into DataFrames.
- **Maps Columns**: Renames source columns to match the target based on your choices.
- **Appends Data**: Combines the data and writes it back to the target file.
- **Returns Stats**: Tells you how many columns and rows were processed.

In [ ]:
def append_data(source_file, target_file, manual_mappings):
    """
    Append data from source Excel file to target Excel file using specified column mappings.
    
    Args:
        source_file (str): Path to the source Excel file
        target_file (str): Path to the target Excel file
        manual_mappings (dict): Dictionary of {target_column: source_column} mappings
    
    Returns:
        dict: Statistics about the operation including column and row counts
    
    Raises:
        Various exceptions if file access or data processing fails
    """
    # Step 1: Read both Excel files into pandas DataFrames
    source_df = pd.read_excel(source_file)
    target_df = pd.read_excel(target_file)

    # Step 2: Create a copy of source data to apply mappings
    mapped_source_df = source_df.copy()
    
    # Apply column mappings by renaming source columns to match target columns
    for target_col, source_col in manual_mappings.items(): # Example manual_mappings: {'Name': 'Full Name', 'Email': 'Email Address'}
        if source_col in mapped_source_df.columns: # Only rename columns that exist in the source data
            mapped_source_df = mapped_source_df.rename(columns={source_col: target_col}) # Rename source column to target column

    # Step 3: Find columns that exist in both dataframes after mapping
    common_columns = list(set(mapped_source_df.columns) & set(target_df.columns))

    # Step 4: Clean data - Fill NaN values with empty strings to avoid errors
    for col in common_columns:
        mapped_source_df[col] = mapped_source_df[col].fillna('') 

    # Step 5: Append data - only using columns that exist in the target file
    appended_df = pd.concat([target_df, mapped_source_df[common_columns]], ignore_index=True)

    # Step 6: Save the combined data back to the target file
    appended_df.to_excel(target_file, index=False)

    # Step 7: Return statistics about the operation
    return {
        'source_columns': len(source_df.columns),
        'target_columns': len(target_df.columns),
        'mapped_columns': len(common_columns),
        'appended_rows': len(mapped_source_df)
    }

## Step 3: Build the Interactive User Interface

Now, we create the UI using `ipywidgets` and `ipyfilechooser`.

### Interface Components
- **File Choosers**: Pick your source and target Excel files.
- **Map Button**: Opens the column mapping interface.
- **Mapping Interface** (appears after clicking 'Map Columns'):
  - Dropdowns for each target column to pick a source column.
  - Suggestions if no exact match is found.
  - A 'Proceed' button to append the data.

In [ ]:
# Create file choosers
source_chooser = FileChooser()
source_chooser.title = 'Select Source Excel File'
target_chooser = FileChooser()
target_chooser.title = 'Select Target Excel File'

# Create map button
map_button = widgets.Button(description='Map Columns')

# Define the mapping function
def on_map_button_clicked(b):
    source_file = source_chooser.selected # Get the selected source file
    target_file = target_chooser.selected # Get the selected target file
    if not source_file or not target_file: # Check if both files are selected
        print('Please select both source and target files.')
        return
    # Load column names (nrows=0 for headers only)
    source_df = pd.read_excel(source_file, nrows=0) # Load the source file to get column names
    target_df = pd.read_excel(target_file, nrows=0) # Load the target file to get column names
    source_columns = source_df.columns.tolist() # Get the column names as a list for the source file
    target_columns = target_df.columns.tolist() # Get the column names as a list for the target file

    # Create mapping widgets
    mapping_widgets = [] # Create an empty list to hold the mapping widgets
    for target_col in target_columns: # Iterate over the target columns
        # Check for exact match
        if target_col in source_columns: # If the target column name is found in the source columns
            default = target_col # Set the default value to the target column name
        else:
            default = 'Not Mapped' # Set the default value to 'Not Mapped'
        # Get fuzzy match suggestions
        matches = process.extract(target_col, source_columns, limit=5) # Get the top 5 fuzzy matches for the target column name
        suggestions = [match[0] for match in matches if match[1] >= 60] # Get suggestions with similarity score of 60% or higher (can be adjusted to get more or less suggestions.)

        # Create dropdown
        dropdown = widgets.Dropdown(
            options=['Not Mapped'] + source_columns, # Include 'Not Mapped' and source column names as options
            value=default, 
            description=target_col + ':'
        )

        # Show suggestions if no exact match
        if default == 'Not Mapped' and suggestions: # If there is no exact match and there are suggestions
            suggestion_label = widgets.Label(value='Suggestions: ' + ', '.join(suggestions)) # Create a label with the suggestions
            mapping_widgets.append(widgets.VBox([dropdown, suggestion_label])) # Add the dropdown and label to the mapping widgets
        else:
            mapping_widgets.append(dropdown) # Add only the dropdown to the mapping widgets

    # Create proceed button
    proceed_button = widgets.Button(description='Proceed')
    # Define what happens when 'Proceed' is clicked
    def on_proceed_button_clicked(b):
        # Collect the manual mappings from the dropdown widgets
        manual_mappings = {}
        for widget in mapping_widgets:  # Assume mapping_widgets is defined elsewhere
            if isinstance(widget, widgets.VBox):
                dropdown = widget.children[0] 
            else:
                dropdown = widget
            target_col = dropdown.description.rstrip(':')
            source_col = dropdown.value
            if source_col != 'Not Mapped':
                manual_mappings[target_col] = source_col
        
        # Append data and provide feedback
        try:
            # Call the append_data function (assumed to be defined elsewhere)
            stats = append_data(source_file, target_file, manual_mappings)
            
            # Display a prominent confirmation message
            display(HTML('<b>Columns have been successfully mapped and data appended.</b>'))
            
            # Print detailed statistics
            print(f'Statistics:\n'
                f'- Source columns: {stats["source_columns"]}\n'
                f'- Target columns: {stats["target_columns"]}\n'
                f'- Mapped columns: {stats["mapped_columns"]}\n'
                f'- Appended rows: {stats["appended_rows"]}')
        
        except Exception as e:
            # Handle any errors with a descriptive message
            print(f'An error occurred while mapping columns or appending data: {str(e)}')
    proceed_button.on_click(on_proceed_button_clicked)
    # Show the mapping interface
    display(widgets.VBox(mapping_widgets + [proceed_button]))

map_button.on_click(on_map_button_clicked)

# Display the interface
display(source_chooser)
display(target_chooser)
display(map_button)